# 海南低空经济3D地图可视化

基于 MapLibre 和 leafmap 实现海南省低空经济产业布局的3D可视化

## 案例说明

本案例参考了 leafmap MapLibre 教程中的两个经典例子：
1. **Vancouver Property Value** - 3D建筑高度可视化技术
2. **3D Indoor Mapping** - 室内3D地图空间层次展示

结合海南省的地理特点和低空经济发展现状，创建了一个综合性的3D可视化案例。

## 安装依赖

In [ ]:
# 安装必要的包
# %pip install "leafmap[maplibre]" pandas numpy

## 导入库

In [1]:
import leafmap.maplibregl as leafmap
import pandas as pd
import json
import math
from typing import Dict, List, Any

print("库导入完成")

库导入完成


## 基础数据配置

In [2]:
# 海南省主要城市坐标
HAINAN_CITIES = {
    "海口": [110.3312, 20.0311],
    "三亚": [109.5146, 18.2577], 
    "儋州": [109.5765, 19.5175],
    "琼海": [110.4667, 19.2463],
    "文昌": [110.7530, 19.5430],
    "万宁": [110.3893, 18.7962],
    "东方": [108.6340, 19.1019],
    "五指山": [109.5169, 18.7669]
}

# 低空经济产业类型配色方案
INDUSTRY_COLORS = {
    "无人机制造": "#FF6B6B",
    "通用航空": "#4ECDC4",
    "航空物流": "#45B7D1",
    "航空旅游": "#96CEB4",
    "航空培训": "#FFEAA7",
    "航空维修": "#DDA0DD",
    "空中救援": "#FFA07A",
    "农业植保": "#98D8C8"
}

print(f"配置了 {len(HAINAN_CITIES)} 个主要城市")
print(f"定义了 {len(INDUSTRY_COLORS)} 种产业类型")

配置了 8 个主要城市
定义了 8 种产业类型


## 数据生成函数

In [3]:
def create_hainan_low_altitude_data():
    """创建海南低空经济模拟数据"""
    
    cities_data = [
        {
            "name": "海口",
            "coordinates": HAINAN_CITIES["海口"],
            "industry_type": "通用航空",
            "investment_amount": 50000,
            "company_count": 25,
            "employment": 1200,
            "flight_volume": 8000,
            "description": "海南省低空经济总部基地，通用航空产业集群"
        },
        {
            "name": "三亚",
            "coordinates": HAINAN_CITIES["三亚"],
            "industry_type": "航空旅游",
            "investment_amount": 35000,
            "company_count": 18,
            "employment": 800,
            "flight_volume": 12000,
            "description": "低空旅游示范区，海上观光航线"
        },
        {
            "name": "文昌",
            "coordinates": HAINAN_CITIES["文昌"],
            "industry_type": "无人机制造",
            "investment_amount": 28000,
            "company_count": 15,
            "employment": 600,
            "flight_volume": 5000,
            "description": "航天城配套无人机产业园"
        },
        {
            "name": "琼海",
            "coordinates": HAINAN_CITIES["琼海"],
            "industry_type": "航空培训",
            "investment_amount": 22000,
            "company_count": 12,
            "employment": 450,
            "flight_volume": 6000,
            "description": "博鳌低空飞行培训基地"
        },
        {
            "name": "儋州",
            "coordinates": HAINAN_CITIES["儋州"],
            "industry_type": "农业植保",
            "investment_amount": 18000,
            "company_count": 20,
            "employment": 350,
            "flight_volume": 15000,
            "description": "农业无人机植保服务中心"
        },
        {
            "name": "万宁",
            "coordinates": HAINAN_CITIES["万宁"],
            "industry_type": "航空物流",
            "investment_amount": 15000,
            "company_count": 8,
            "employment": 280,
            "flight_volume": 4000,
            "description": "海岛物流无人机配送网络"
        },
        {
            "name": "东方",
            "coordinates": HAINAN_CITIES["东方"],
            "industry_type": "空中救援",
            "investment_amount": 12000,
            "company_count": 6,
            "employment": 200,
            "flight_volume": 2000,
            "description": "应急救援航空基地"
        },
        {
            "name": "五指山",
            "coordinates": HAINAN_CITIES["五指山"],
            "industry_type": "航空维修",
            "investment_amount": 10000,
            "company_count": 5,
            "employment": 150,
            "flight_volume": 1500,
            "description": "山区航空器维修保障中心"
        }
    ]
    
    features = []
    for city in cities_data:
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": city["coordinates"]
            },
            "properties": {
                "name": city["name"],
                "industry_type": city["industry_type"],
                "investment_amount": city["investment_amount"],
                "company_count": city["company_count"],
                "employment": city["employment"],
                "flight_volume": city["flight_volume"],
                "description": city["description"],
                "color": INDUSTRY_COLORS[city["industry_type"]],
                "height": city["investment_amount"] / 100,
                "base_height": 0
            }
        }
        features.append(feature)
    
    return {
        "type": "FeatureCollection",
        "features": features
    }

print("产业数据生成函数定义完成")

产业数据生成函数定义完成


In [4]:
def create_flight_routes_data():
    """创建海南低空飞行航线数据"""
    
    routes = [
        {"from": "海口", "to": "三亚", "type": "旅游航线", "frequency": 50},
        {"from": "海口", "to": "文昌", "type": "货运航线", "frequency": 30},
        {"from": "三亚", "to": "万宁", "type": "观光航线", "frequency": 25},
        {"from": "海口", "to": "琼海", "type": "培训航线", "frequency": 20},
        {"from": "儋州", "to": "东方", "type": "农业航线", "frequency": 40},
        {"from": "海口", "to": "儋州", "type": "综合航线", "frequency": 35},
        {"from": "三亚", "to": "五指山", "type": "救援航线", "frequency": 15}
    ]
    
    features = []
    for route in routes:
        from_coord = HAINAN_CITIES[route["from"]]
        to_coord = HAINAN_CITIES[route["to"]]
        
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": [from_coord, to_coord]
            },
            "properties": {
                "from_city": route["from"],
                "to_city": route["to"],
                "route_type": route["type"],
                "frequency": route["frequency"],
                "width": route["frequency"] / 10
            }
        }
        features.append(feature)
    
    return {
        "type": "FeatureCollection", 
        "features": features
    }

print("航线数据生成函数定义完成")

航线数据生成函数定义完成


## 创建3D地图

In [5]:
# 生成数据
industry_data = create_hainan_low_altitude_data()
routes_data = create_flight_routes_data()

print("数据生成完成")
print(f"产业园区数量: {len(industry_data['features'])}")
print(f"航线数量: {len(routes_data['features'])}")

数据生成完成
产业园区数量: 8
航线数量: 7


In [6]:
# 创建地图
m = leafmap.Map(
    center=[109.8, 19.2],
    zoom=8,
    pitch=45,
    bearing=0,
    style="dark-matter"
)

print("基础地图创建完成")
m

基础地图创建完成


Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

In [7]:
# 创建3D柱状图数据
def create_industry_circles(industry_data):
    industry_circles = {
        "type": "FeatureCollection",
        "features": []
    }
    
    for feature in industry_data["features"]:
        coord = feature["geometry"]["coordinates"]
        props = feature["properties"]
        
        # 创建圆形多边形
        radius = 0.05
        points = []
        for i in range(16):
            angle = 2 * math.pi * i / 16
            lon = coord[0] + radius * math.cos(angle)
            lat = coord[1] + radius * math.sin(angle)
            points.append([lon, lat])
        points.append(points[0])
        
        circle_feature = {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [points]
            },
            "properties": props
        }
        industry_circles["features"].append(circle_feature)
    
    return industry_circles

industry_circles = create_industry_circles(industry_data)

# 添加3D柱状图
paint_industry_3d = {
    "fill-extrusion-color": ["get", "color"],
    "fill-extrusion-height": ["get", "height"],
    "fill-extrusion-base": ["get", "base_height"],
    "fill-extrusion-opacity": 0.8
}

m.add_geojson(
    industry_circles,
    layer_type="fill-extrusion",
    paint=paint_industry_3d,
    name="industry-3d"
)

print("产业园区3D柱状图添加完成")
m

产业园区3D柱状图添加完成


Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

In [11]:
# 添加航线
paint_routes = {
    "line-color": "#00FFFF",
    "line-width": ["get", "width"],
    "line-opacity": 0.8
}

m.add_geojson(
    routes_data,
    layer_type="line", 
    paint=paint_routes,
    name="flight-routes"
)

print("飞行航线网络添加完成")
m

飞行航线网络添加完成


Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

In [9]:
# 添加城市标记
paint_cities = {
    "circle-color": "#FFFFFF",
    "circle-radius": 8,
    "circle-stroke-color": "#000000",
    "circle-stroke-width": 2,
    "circle-opacity": 0.9
}

m.add_geojson(
    industry_data,
    layer_type="circle",
    paint=paint_cities,
    name="cities"
)

print("城市标记点添加完成")
print("海南低空经济3D地图创建完成！")
m

城市标记点添加完成
海南低空经济3D地图创建完成！


Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

## 数据分析

In [10]:
# 统计分析
print("海南低空经济产业统计")
print("=" * 50)

total_investment = 0
total_companies = 0
total_employment = 0
total_flights = 0

for feature in industry_data["features"]:
    props = feature["properties"]
    city = props["name"]
    industry = props["industry_type"]
    investment = props["investment_amount"]
    companies = props["company_count"]
    employment = props["employment"]
    flights = props["flight_volume"]
    
    total_investment += investment
    total_companies += companies
    total_employment += employment
    total_flights += flights
    
    print(f"{city:6} | {industry:8} | 投资:{investment:5}万 | 企业:{companies:2}家 | 就业:{employment:4}人 | 飞行:{flights:5}架次")

print("-" * 50)
print(f"总计: 投资 {total_investment:,} 万元 | 企业 {total_companies} 家 | 就业 {total_employment:,} 人 | 飞行 {total_flights:,} 架次")

海南低空经济产业统计
海口     | 通用航空     | 投资:50000万 | 企业:25家 | 就业:1200人 | 飞行: 8000架次
三亚     | 航空旅游     | 投资:35000万 | 企业:18家 | 就业: 800人 | 飞行:12000架次
文昌     | 无人机制造    | 投资:28000万 | 企业:15家 | 就业: 600人 | 飞行: 5000架次
琼海     | 航空培训     | 投资:22000万 | 企业:12家 | 就业: 450人 | 飞行: 6000架次
儋州     | 农业植保     | 投资:18000万 | 企业:20家 | 就业: 350人 | 飞行:15000架次
万宁     | 航空物流     | 投资:15000万 | 企业: 8家 | 就业: 280人 | 飞行: 4000架次
东方     | 空中救援     | 投资:12000万 | 企业: 6家 | 就业: 200人 | 飞行: 2000架次
五指山    | 航空维修     | 投资:10000万 | 企业: 5家 | 就业: 150人 | 飞行: 1500架次
--------------------------------------------------
总计: 投资 190,000 万元 | 企业 109 家 | 就业 4,030 人 | 飞行 53,500 架次


## 使用说明

### 地图操作
- **拖拽**: 平移地图视角
- **滚轮**: 缩放地图
- **Shift + 拖拽**: 旋转地图
- **Ctrl + 拖拽**: 调整俯仰角度

### 图层说明
1. **3D柱状图**: 各城市低空经济产业园区，高度代表投资金额
2. **青色线条**: 主要飞行航线，线宽代表航线频次
3. **白色圆点**: 城市位置标记

### 技术特点
- 基于 MapLibre 和 leafmap 实现
- 参考 Vancouver Property Value 的3D柱状图技术
- 支持交互式3D浏览
- 完全开源，无需外部API密钥